In [ ]:
# --- STEP 1: INSTALL LIBRARIES ---
# !pip install gradio gradio_client pillow -q

# --- STEP 2: IMPORT TOOLS ---
import gradio as gr
from gradio_client import Client, handle_file
from PIL import Image
import time

# ==========================================
# 👇 YOUR TOKEN IS SET BELOW 👇
HF_TOKEN = "Your_Hugging_Face_Token"
# ==========================================

# --- STEP 3: HELPER FUNCTION (PREVENTS STRETCHING) ---
def resize_with_padding(image_path, target_size=(768, 1024)):
    """Resizes image to 3:4 ratio with black bars to prevent distortion"""
    if not image_path: return None
    img = Image.open(image_path)
    img.thumbnail((target_size[0], target_size[1]), Image.Resampling.LANCZOS)

    new_img = Image.new("RGB", target_size, (0, 0, 0))
    left = (target_size[0] - img.width) // 2
    top = (target_size[1] - img.height) // 2
    new_img.paste(img, (left, top))

    save_path = f"temp_person_{int(time.time())}.jpg"
    new_img.save(save_path)
    return save_path

# --- STEP 4: THE CORE AI FUNCTION ---
def virtual_tryon(person_image, cloth_image, cloth_desc):
    # 1. Validation
    if person_image is None or cloth_image is None:
        return None, "⚠️ Error: Please upload both the Person and the Cloth."

    # 2. Resize the person (Crucial for good results)
    fixed_person_path = resize_with_padding(person_image)

    print(f"--> Sending to AI: '{cloth_desc}'")

    try:
        # 3. Connect to the AI
        client = Client("yisol/IDM-VTON", hf_token=HF_TOKEN)

        # 4. Generate the Image
        result = client.predict(
            dict={"background": handle_file(fixed_person_path), "layers": [], "composite": None},
            garm_img=handle_file(cloth_image),
            garment_des=cloth_desc,  # "Red t-shirt", "Denim jacket", etc.
            is_checked=True,         # Auto-masking (Find the body automatically)
            is_checked_crop=False,   # We handled cropping ourselves
            denoise_steps=30,        # Good balance of speed vs quality
            seed=42,
            api_name="/tryon"
        )

        # 5. Return Success
        return result[0], "✅ Success! Try-on complete."

    except Exception as e:
        # Handle errors (like Queue full)
        return None, f"❌ Error: {e}"

# --- STEP 5: BUILD THE UI ---
with gr.Blocks(theme=gr.themes.Ocean()) as app:
    gr.Markdown("# 👕 AI Virtual Try-On (Tops Only)")
    gr.Markdown("### Upload a photo of yourself and a Top (Shirt, T-Shirt, Jacket).")

    with gr.Row():
        with gr.Column():
            # Input 1: The Person (Allows Webcam OR File Upload)
            input_person = gr.Image(label="1. Your Photo", type="filepath", sources=["upload", "webcam"])

            # Input 2: The Cloth (File Upload Only)
            input_cloth = gr.Image(label="2. The Cloth (Shirt/T-Shirt)", type="filepath", sources=["upload"])

            # Input 3: Description (Helps the AI)
            input_desc = gr.Textbox(label="3. Describe the cloth", value="A cool shirt", placeholder="e.g. Red polo shirt")

            btn = gr.Button("🪄 Try it On!", variant="primary")

        with gr.Column():
            output_image = gr.Image(label="Result")
            status_box = gr.Textbox(label="Status")

    # Connect the button to the function
    btn.click(
        fn=virtual_tryon,
        inputs=[input_person, input_cloth, input_desc],
        outputs=[output_image, status_box]
    )

# --- STEP 6: LAUNCH ---
if __name__ == "__main__":
    app.launch(share=True, debug=True)

/tmp/ipython-input-2284549352.py:66: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Ocean()) as app:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1fdcb99d48afd54272.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


--> Sending to AI: 'A cool shirt'
Loaded as API: https://yisol-idm-vton.hf.space ✔


/tmp/ipython-input-2284549352.py:44: DeprecationWarning: The 'hf_token' parameter will be removed in Gradio 6.0. You will need to use 'token' instead.
  client = Client("yisol/IDM-VTON", hf_token=HF_TOKEN)


--> Sending to AI: 'A cool shirt'


/tmp/ipython-input-2284549352.py:44: DeprecationWarning: The 'hf_token' parameter will be removed in Gradio 6.0. You will need to use 'token' instead.
  client = Client("yisol/IDM-VTON", hf_token=HF_TOKEN)


Loaded as API: https://yisol-idm-vton.hf.space ✔
